In [1]:
import numpy as np, matplotlib.pyplot as plt, matplotlib as mpl
import pcmdpy_gpu as ppy
import pandas as pd
from tqdm import tqdm
from importlib import util
from seaborn.apionly import color_palette
%matplotlib inline

/n/home01/bcook/.conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/n/home01/bcook/.conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)
/n/home01/bcook/.conda/envs/py36/lib/python3.6/site-packages/seaborn/apionly.py:9: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [2]:
plt.style.use('cook')

In [3]:
from setup_files.mocks_paper1.mock_models import models, run_names, results

/n/home01/bcook/pCMDs/pixcmd/pcmdpy/pcmdpy/plotting/results.py:83: RuntimeWarning: invalid value encountered in sqrt
  logzerrs.append(np.sqrt(logzvar))
/n/home01/bcook/.local/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


# Setup

In [27]:
known_keys = ['use_gpu', 'verbose', 'dynamic', 'Nim', 'filters', 'iso_model', 'gal_model', 'like_mode', 'bins', 'downsample', 'mag_system', 'lum_cut', 'fixed_seed', 'sky_noise', 'shot_noise', 'prior', 'data_is_mock', 'data_pcmd']

In [13]:
f_mock = ppy.instrument.default_m31_filters()

In [14]:
iso_model = ppy.isochrones.Isochrone_Model(f_mock, mag_system='vega')

In [15]:
driv = ppy.driver.Driver(iso_model, gpu=True)

Please wait while the GPU states are initialized

In [30]:
def get_max_logl(key, n_samples=100, verbose=True):
    config_mod = 'setup_files/mocks_paper1'
    config_file = f'setup_files/mocks_paper1/{key}.py'
    spec = util.spec_from_file_location(config_mod, config_file)
    config = util.module_from_spec(spec)
    spec.loader.exec_module(config)
    params = config.params
    
    for k in params.keys():
        assert (k in known_keys), f"Key unknown: {k}"
    assert params['data_is_mock'], "Have not implemented data runs yet"
    
    driv.filters = params['filters']
    driv.iso_model = params['iso_model']
    driv.initialize_data(params['data_pcmd'], bins=params['bins'])
    best_params = results[key].best_params
    Nim = params['Nim']
    gal_model = params['gal_model']
    prior = params.get('prior', gal_model.get_flat_prior())
    
    logl_kwargs = {'like_mode': params['like_mode'],
                  'downsample': params['downsample'],
                  'mag_system': params['mag_system'],
                  'sky_noise': params['sky_noise'],
                  'shot_noise': params['shot_noise']}
    
    logls = [ppy.fit_model.lnlike(best_params, driv, Nim, prior.lnprior, gal_model, fixed_seed=False,
                                 **logl_kwargs) for _ in tqdm(range(n_samples), disable=(not verbose))]
    return np.median(logls)

In [61]:
def add_max_logl(key, max_logl):
    filename = f'results/paper1_{key}.csv'
    with open(filename, 'r') as f:
        text = f.read()
        if 'max_logl' in text:
            text = text.partition('\n')[-1]
    with open(filename, 'w') as f:
        f.write(f'# max_logl : {max_logl:.5e}\n')
        f.write(text)

In [11]:
still_running = ['mock_8', 'mock_11', 'mock_43', 'mock_44', 'mock_45', 'mock_46', 'mock_47', 'mock_48', 'mock_49', 'mock_50', 'mock_51', 'mock_52', 'mock_53', 'mock_54',
            'mismatch_2', 'mismatch_5', 'mismatch_8', 'mismatch_9']

In [ ]:
max_logls = {}
for k in tqdm(list(results.keys())):
    if k in still_running:
        continue
    max_logl = get_max_logl(k, n_samples=100, verbose=False)
    max_logls[k] = max_logl
    print(k, max_logl)
    add_max_logl(k, max_logl)

In [10]:
magbins = np.arange(10, 45, 0.05)
colorbins = np.arange(-1.5, 5.6, 0.05)  # fairly insensitive to distance
bins = [magbins, colorbins]

magbins_n = np.arange(10, 45, 0.02)
colorbins_n = np.arange(-1.5, 5.6, 0.02)  # fairly insensitive to distance
narrow_bins = [magbins_n, colorbins_n]

magbins_w = np.arange(10, 45, 0.1)
colorbins_w = np.arange(-1.5, 5.6, 0.1)  # fairly insensitive to distance
wide_bins = [magbins_w, colorbins_w]

In [11]:
logls = {}
logls_true = {}

In [12]:
N_sample = 10
for k, r in tqdm(results.items()):
    season = getattr(r, 'season', '')
    if 'summer' in season:
        driv = driv_summer
    elif 'winter' in season:
        driv = driv_winter
    else:
        driv = driv_mock
    Nim_mock = 256
    Nim = 512
    if k == 'mock_10':
        Nim = 256
    if k == 'mock_11':
        Nim = 1024
    like_mode = 2
    if k == 'mock_9':
        like_mode = 3
    these_bins = bins
    if k in ['mock_29', 'mock_31']:
        these_bins = wide_bins
    elif k in ['mock_30', 'mock_32']:
        these_bins = narrow_bins
    logl = []
    logl_true = []
    pcmd_orig = driv.simulate(r.best_model, Nim_mock, fixed_seed=True)[0]
    if r.true_model is not None:
        pcmd_true = driv.simulate(r.true_model, Nim_mock, fixed_seed=True)[0]
    for _ in range(N_sample):
        pcmd_new = driv.simulate(r.best_model, Nim)[0]
        driv.initialize_data(pcmd_orig, bins=these_bins)
        logl.append(driv.loglike(pcmd_new, like_mode=like_mode))
        if r.true_model is not None:
            driv.initialize_data(pcmd_true, bins=these_bins)
            logl_true.append(driv.loglike(pcmd_new, like_mode=like_mode))
    logls[k] = logl
    logls_true[k] = logl_true
print(logls)
print(logls_true)

100%|██████████| 126/126 [35:41<00:00, 14.40s/it]

{'mock_1': [-941.2131331266269, -921.5289646902845, -904.8413566671223, -1024.8324814687885, -943.1232729392, -1025.70857319633, -932.7179923382454, -959.5151413734507, -976.4811526061031, -983.2968065386245], 'mock_2': [-48.740010291819985, -49.657400698497206, -64.01566453756702, -82.39651895955326, -63.210677401151564, -63.429834831173196, -50.63332694966924, -47.12297571268363, -51.720749737722805, -66.04362690795506], 'mock_3': [-900.8330234626495, -930.4548328806753, -950.6661072872458, -904.2736035897037, -918.7363400491737, -923.702275279181, -900.1914003137883, -934.4664981646351, -1011.2103862801366, -1001.0805437105365], 'mock_4': [-69.30651345059219, -45.494945050044265, -50.100166292474704, -48.17034034373262, -43.35241247548609, -56.23325440306603, -60.22654335862295, -51.51130050879049, -53.16394961844399, -44.67941301250217], 'mock_5': [-940.4421496713054, -985.0672086036722, -975.5544511088705, -992.120893463221, -1059.476269085692, -967.4032616980692, -972.67255709464

In [13]:
logl_means = {k: np.mean(v) for k,v in logls.items()}
logl_true_means = {k: np.mean(v) for k,v in logls_true.items()}

/n/home01/bcook/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/n/home01/bcook/.local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [14]:
df_logl = pd.DataFrame.from_dict(logl_means, 'index', columns=['logl_mean'])
df_true = pd.DataFrame.from_dict(logl_true_means, 'index', columns=['logl_true'])
df_logl = df_logl.merge(df_true, how='outer', left_index=True, right_index=True)
df_logl.head()

,logl_mean,logl_true
mock_1,-961.325887,-1050.856834
mock_2,-58.697079,-77.129775
mock_3,-937.561501,-946.515603
mock_4,-52.223884,-416.995630
mock_5,-984.609334,-995.698322


In [15]:
df_mock = df_logl[~np.in1d(df_logl.index.values, list(m31_res.keys()))]
df_m31 = df_logl[np.in1d(df_logl.index.values, list(m31_res.keys()))]
df_m31.drop('logl_true', axis=1, inplace=True)

/n/home01/bcook/.conda/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [18]:
df_mock

,logl_mean,logl_true
mock_1,-961.325887,-1050.856834
mock_2,-58.697079,-77.129775
mock_3,-937.561501,-946.515603
mock_4,-52.223884,-416.995630
mock_5,-984.609334,-995.698322
mock_6,-956.931901,-1026.894345
mock_7,-981.360851,-1229.886050
mock_8,-985.698394,-1215.054688
mock_9,928948.080988,928914.186562
mock_10,-917.420752,-935.719779


In [16]:
df_mock.to_csv('setup_files/mocks_paper1/max_logls.csv', float_format='%.3f', index_label='run_name')

In [17]:
df_m31.to_csv('setup_files/m31_paper2/max_logls.csv', float_format='%.3f', index_label='run_name')